<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [234]:
!pip install sqlalchemy==2.0

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [235]:
!pip install ipython-sql
!pip install ipython-sql prettytable

In [236]:
# Cargar extensión SQL de Jupyter
#%load_ext sql
%reload_ext sql

# Librerías necesarias
import sqlite3
import pandas as pd
import prettytable
prettytable.DEFAULT = 'DEFAULT'


In [237]:
# Conectarse a base de datos (puedes cambiar el nombre si hay errores de solo lectura)
con = sqlite3.connect("temporary.db")
cur = con.cursor()

In [238]:
# Establecer conexión con %sql magic
%sql sqlite:///temporary.db

In [261]:
# Cargar dataset original desde la URL
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Renombrar la columna 'PAYLOAD_MASS__KG_' a 'Payload Mass (kg)'
df.rename(columns={'PAYLOAD_MASS__KG_': 'Payload Mass (kg)'}, inplace=True)

# Guardar DataFrame como tabla SQL
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

In [262]:
# Verificar filas en tabla original
cursor = con.cursor()
cursor.execute("SELECT COUNT(*) FROM SPACEXTBL")
print("Filas en SPACEXTBL:", cursor.fetchone()[0])

Filas en SPACEXTBL: 101


In [263]:
# Eliminar tabla anterior si existe
cursor.execute("DROP TABLE IF EXISTS SPACEXTABLE")

# Crear nueva tabla filtrando los nulos
cursor.execute("CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL")

# Confirmar cuántas filas tiene la nueva tabla
cursor.execute("SELECT COUNT(*) FROM SPACEXTABLE")
print("Filas en SPACEXTABLE:", cursor.fetchone()[0])

Filas en SPACEXTABLE: 101


**Note:This below code is added to remove blank rows from table**


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [264]:
%sql SELECT DISTINCT "Launch Site" FROM SPACEXTABLE;

   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[('Launch Site',)]


### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [265]:
%%sql
SELECT *
FROM SPACEXTABLE
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5;

   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[('2010-06-04', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)'),
 ('2010-12-08', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)'),
 ('2012-05-22', '7:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt'),
 ('2012-10-08', '0:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', 500, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt'),
 ('2013-03-01', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', 677, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')]

### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [266]:
%%sql 
SELECT SUM("Payload Mass (kg)") AS total_payload_mass
FROM SPACEXTABLE
WHERE Customer LIKE '%NASA (CRS)%';

   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[(48213,)]

### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [269]:
%%sql
SELECT AVG("Payload Mass (kg)") AS average_payload_mass
FROM SPACEXTABLE
WHERE "Booster_Version" = 'F9 v1.1';

   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[(2928.4,)]

### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [271]:
%%sql
SELECT MIN(Date) AS first_successful_ground_landing_date
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (ground pad)';

   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[('2015-12-22',)]

### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [273]:
%%sql
SELECT DISTINCT "Booster_Version"
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (drone ship)'
AND "Payload Mass (kg)" > 4000
AND "Payload Mass (kg)" < 6000;


   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[('F9 FT B1022',), ('F9 FT B1026',), ('F9 FT  B1021.2',), ('F9 FT  B1031.2',)]

### Task 7




##### List the total number of successful and failure mission outcomes


In [248]:
%%sql 
SELECT "Mission_Outcome", COUNT(*) AS total_missions
FROM SPACEXTABLE
GROUP BY "Mission_Outcome";


   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[('Failure (in flight)', 1),
 ('Success', 98),
 ('Success ', 1),
 ('Success (payload status unclear)', 1)]

### Task 8



##### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.


In [249]:
%%sql
SELECT DISTINCT "Booster Version"
FROM SPACEXTABLE
WHERE "Payload Mass (kg)" = (SELECT MAX("Payload Mass (kg)") FROM SPACEXTABLE);


   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[('Booster Version',)]

### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [250]:
%%sql
SELECT substr(Date, 6, 2) AS month,
           "Landing _Outcome",
           "Booster Version",
           "Launch Site"
FROM SPACEXTABLE
WHERE "Landing _Outcome" LIKE 'Failure (drone ship)'
AND substr(Date, 0, 5) = '2015';


   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[]

### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [251]:
%%sql
SELECT "Landing _Outcome", COUNT(*) AS outcome_count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing _Outcome"
ORDER BY outcome_count DESC;


   sqlite:///my_data1.db
 * sqlite:///temporary.db
Done.


[('Landing _Outcome', 31)]

### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
